Segmenting and Clustering Neighborhoods in Toronto - Part **1**

Introduction

In this assignment, you will be required to explore and cluster the neighborhoods in Toronto. You will group the neighborhoods into clusters. In this part, we download the dataset in a dataframe to store the data to be explored in the next parts.

Before we get the data and start exploring it, let's download all the dependencies that we will need.
1. Import libraries

In [14]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


2. Scrap data from Wikipedia page into a DataFrame

In [16]:
# send the GET request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [17]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [18]:
# create three lists to store table data
postalCodeList = []
boroughList = []
neighborhoodList = []

Using BeautifulSoup

In [20]:
# find the table
soup.find('table').find_all('tr')

# find all the rows of the table
soup.find('table').find_all('tr')

# for each row of the table, find all the table data
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')

In [21]:
# append the data into the respective lists
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n')) # avoid new lines in neighborhood cell

In [22]:
# create a new DataFrame from the three lists
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

toronto_df.head()

,PostalCode,Borough,Neighborhood
0,\nM1ANot assigned\n\n,\nM2ANot assigned\n\n,\nM3ANorth York(Parkwoods)
1,\nM1BScarborough(Malvern / Rouge)\n\n,\nM2BNot assigned\n\n,\nM3BNorth York(Don Mills)North
2,\nM1CScarborough(Rouge Hill / Port Union / Hig...,\nM2CNot assigned\n\n,\nM3CNorth York(Don Mills)South(Flemingdon Park)
3,\nM1EScarborough(Guildwood / Morningside / Wes...,\nM2ENot assigned\n\n,\nM3ENot assigned
4,\nM1GScarborough(Woburn)\n\n,\nM2GNot assigned\n\n,\nM3GNot assigned


3. Drop cells with a borough that is "Not assigned"

In [23]:
# drop cells with a borough that is Not assigned
toronto_df_dropna = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_dropna.head()

,PostalCode,Borough,Neighborhood
0,\nM1ANot assigned\n\n,\nM2ANot assigned\n\n,\nM3ANorth York(Parkwoods)
1,\nM1BScarborough(Malvern / Rouge)\n\n,\nM2BNot assigned\n\n,\nM3BNorth York(Don Mills)North
2,\nM1CScarborough(Rouge Hill / Port Union / Hig...,\nM2CNot assigned\n\n,\nM3CNorth York(Don Mills)South(Flemingdon Park)
3,\nM1EScarborough(Guildwood / Morningside / Wes...,\nM2ENot assigned\n\n,\nM3ENot assigned
4,\nM1GScarborough(Woburn)\n\n,\nM2GNot assigned\n\n,\nM3GNot assigned


4. Group neighborhoods in the same borough

In [24]:
# group neighborhoods in the same borough
toronto_df_grouped = toronto_df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,\nM1ANot assigned\n\n,\nM2ANot assigned\n\n,\nM3ANorth York(Parkwoods)
1,\nM1BScarborough(Malvern / Rouge)\n\n,\nM2BNot assigned\n\n,\nM3BNorth York(Don Mills)North
2,\nM1CScarborough(Rouge Hill / Port Union / Hig...,\nM2CNot assigned\n\n,\nM3CNorth York(Don Mills)South(Flemingdon Park)
3,\nM1EScarborough(Guildwood / Morningside / Wes...,\nM2ENot assigned\n\n,\nM3ENot assigned
4,\nM1GScarborough(Woburn)\n\n,\nM2GNot assigned\n\n,\nM3GNot assigned


5. For Neighborhood="Not assigned", make the value the same as Borough

In [25]:
# for Neighborhood="Not assigned", make the value the same as Borough
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,\nM1ANot assigned\n\n,\nM2ANot assigned\n\n,\nM3ANorth York(Parkwoods)
1,\nM1BScarborough(Malvern / Rouge)\n\n,\nM2BNot assigned\n\n,\nM3BNorth York(Don Mills)North
2,\nM1CScarborough(Rouge Hill / Port Union / Hig...,\nM2CNot assigned\n\n,\nM3CNorth York(Don Mills)South(Flemingdon Park)
3,\nM1EScarborough(Guildwood / Morningside / Wes...,\nM2ENot assigned\n\n,\nM3ENot assigned
4,\nM1GScarborough(Woburn)\n\n,\nM2GNot assigned\n\n,\nM3GNot assigned


6. Check whether it is the same as required by the question

In [26]:
# create a new test dataframe
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_grouped[toronto_df_grouped["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood


7. Finally, print the number of rows of the cleaned dataframe

In [27]:
# print the number of rows of the cleaned dataframe
toronto_df_grouped.shape

(20, 3)